In [ ]:
%%html
<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at 'In [13]'.</span>

An Exception was encountered at 'In [13]'.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob
import nibabel as nib
import os
import time

import pandas as pd
import numpy as np

from mricode.utils import log_textfile
from mricode.utils import copy_colab
from mricode.utils import return_iter
from mricode.utils import return_csv

from mricode.models.SimpleCNN import SimpleCNN
from mricode.models.DenseNet import MyDenseNet

import tensorflow as tf
from tensorflow.keras.layers import Conv3D
from tensorflow import nn
from tensorflow.python.ops import nn_ops
from tensorflow.python.framework import tensor_shape
from tensorflow.python.keras.engine.base_layer import InputSpec
from tensorflow.python.keras.utils import conv_utils

tf.__version__

'2.0.0'

In [3]:
tf.test.is_gpu_available()

True

In [4]:
path_saved = './modelssaved/DenseNet_T1T2_site16_norm_20e_SimpleCNN/'
path_output = './output/'
path_tfrecords = '/data2/res64/down/'
path_csv = '/data2/csv/'
filename_res = {'train': 'intell_residual_train.csv', 'val': 'intell_residual_valid.csv', 'test': 'intell_residual_test.csv'}
filename_final = filename_res
sample_size = 'site16_allimages'
batch_size = 8
onlyt1 = False
modelname = 'runAllImages64_DenseNet_T1T2_site16_norm_20e_SimpleCNN_'
Model = SimpleCNN
t1_mean=1.3779395849814497
t1_std=3.4895845243139503
t2_mean=2.22435586968901
t2_std=5.07708743178319
ad_mean=1.3008901218593748e-05
ad_std=0.009966655860940228
fa_mean=0.0037552628409334037
fa_std=0.012922319568740915
md_mean=9.827903909139596e-06
md_std=0.009956973204022659
rd_mean=8.237404999587111e-06
rd_std=0.009954672598675338

In [5]:
train_iter, val_iter, test_iter = return_iter(path_tfrecords, sample_size, batch_size, onlyt1=onlyt1)

In [6]:
if False:
  t1_mean = 0.
  t1_std = 0.
  t2_mean = 0.
  t2_std = 0.
  ad_mean = 0.
  ad_std = 0.
  fa_mean = 0.
  fa_std = 0.
  md_mean = 0.
  md_std = 0.
  rd_mean = 0.
  rd_std = 0.
  n = 0.
  for b in train_iter:
      t1_mean += np.mean(b['t1'])
      t1_std += np.std(b['t1'])
      t2_mean += np.mean(b['t2'])
      t2_std += np.std(b['t2'])
      a = np.asarray(b['ad'])
      a = a.copy()
      a[np.isnan(a)] = 0
      ad_mean += np.mean(a)
      ad_std += np.std(a)
      a = np.asarray(b['fa'])
      a = a.copy()
      a[np.isnan(a)] = 0
      fa_mean += np.mean(a)
      fa_std += np.std(a)
      a = np.asarray(b['md'])
      a = a.copy()
      a[np.isnan(a)] = 0
      md_mean += np.mean(a)
      md_std += np.std(a)
      a = np.asarray(b['rd'])
      a = a.copy()
      a[np.isnan(a)] = 0
      rd_mean += np.mean(a)
      rd_std += np.std(a)
      n += np.asarray(b['t1']).shape[0]

  t1_mean /= n
  t1_std /= n
  t2_mean /= n
  t2_std /= n
  ad_mean /= n
  ad_std /= n
  fa_mean /= n
  fa_std /= n
  md_mean /= n
  md_std /= n
  rd_mean /= n
  rd_std /= n

In [7]:
t1_mean, t1_std, t2_mean, t2_std, ad_mean, ad_std, fa_mean, fa_std, md_mean, md_std, rd_mean, rd_std

(1.3779395849814497,
 3.4895845243139503,
 2.22435586968901,
 5.07708743178319,
 1.3008901218593748e-05,
 0.009966655860940228,
 0.0037552628409334037,
 0.012922319568740915,
 9.827903909139596e-06,
 0.009956973204022659,
 8.237404999587111e-06,
 0.009954672598675338)

In [8]:
train_df, val_df, test_df, norm_dict = return_csv(path_csv, filename_final, False)

In [9]:
norm_dict

{'BMI': {'mean': 18.681548127052135, 'std': 4.193043131845343},
 'age': {'mean': 119.00325844623563, 'std': 7.479129774017182},
 'height': {'mean': 55.25173666322929, 'std': 3.152756181679028},
 'nihtbx_cardsort_uncorrected': {'mean': 0.01902727147573316,
  'std': 0.92710655806542},
 'nihtbx_cryst_uncorrected': {'mean': 0.007018628014748754,
  'std': 0.7845373584638602},
 'nihtbx_flanker_uncorrected': {'mean': 0.02188780794049048,
  'std': 0.9070917080607726},
 'nihtbx_fluidcomp_uncorrected': {'mean': 0.020178243913565427,
  'std': 0.86606123778624},
 'nihtbx_list_uncorrected': {'mean': 0.00625176016120734,
  'std': 0.8898550695735616},
 'nihtbx_pattern_uncorrected': {'mean': 0.020721412569885096,
  'std': 0.9486618556882954},
 'nihtbx_picture_uncorrected': {'mean': 0.0005782175223803825,
  'std': 0.9577989703304521},
 'nihtbx_picvocab_uncorrected': {'mean': 0.0068509109986280275,
  'std': 0.8038465951211212},
 'nihtbx_reading_uncorrected': {'mean': 0.0027847502208883574,
  'std': 0.85

In [10]:
cat_cols = {'female': 2, 'race.ethnicity': 5, 'high.educ_group': 4, 'income_group': 8, 'married': 6}
num_cols = [x for x in list(val_df.columns) if '_norm' in x]

In [11]:
def calc_loss_acc(out_loss, out_acc, y_true, y_pred, cat_cols, num_cols, norm_dict):
  for col in num_cols:
    tmp_col = col
    tmp_std = norm_dict[tmp_col.replace('_norm','')]['std']
    tmp_y_true = tf.cast(y_true[col], tf.float32).numpy()
    tmp_y_pred = np.squeeze(y_pred[col].numpy())
    if not(tmp_col in out_loss):
      out_loss[tmp_col] = np.sum(np.square(tmp_y_true-tmp_y_pred))
    else:
      out_loss[tmp_col] += np.sum(np.square(tmp_y_true-tmp_y_pred))
    if not(tmp_col in out_acc):
      out_acc[tmp_col] = np.sum(np.square((tmp_y_true-tmp_y_pred)*tmp_std))
    else:
      out_acc[tmp_col] += np.sum(np.square((tmp_y_true-tmp_y_pred)*tmp_std))
  for col in list(cat_cols.keys()):
    tmp_col = col
    if not(tmp_col in out_loss):
      out_loss[tmp_col] = tf.keras.losses.SparseCategoricalCrossentropy()(tf.squeeze(y_true[col]), tf.squeeze(y_pred[col])).numpy()
    else:
      out_loss[tmp_col] += tf.keras.losses.SparseCategoricalCrossentropy()(tf.squeeze(y_true[col]), tf.squeeze(y_pred[col])).numpy()
    if not(tmp_col in out_acc):
      out_acc[tmp_col] = tf.reduce_sum(tf.dtypes.cast((y_true[col] == tf.argmax(y_pred[col], axis=-1)), tf.float32)).numpy()
    else:
      out_acc[tmp_col] += tf.reduce_sum(tf.dtypes.cast((y_true[col] == tf.argmax(y_pred[col], axis=-1)), tf.float32)).numpy()    
  return(out_loss, out_acc)

def format_output(out_loss, out_acc, n, cols, print_bl=False):
  loss = 0
  acc = 0
  output = []
  for col in cols:
    output.append([col, out_loss[col]/n, out_acc[col]/n])
    loss += out_loss[col]/n
    acc += out_acc[col]/n
  df = pd.DataFrame(output)
  df.columns = ['name', 'loss', 'acc']
  if print_bl:
    print(df)
  return(loss, acc, df)

@tf.function
def train_step(X, y, model, optimizer, cat_cols, num_cols):
  with tf.GradientTape() as tape:
    predictions = model(X)
    i = 0
    loss = tf.keras.losses.MSE(tf.cast(y[num_cols[i]], tf.float32), tf.squeeze(predictions[num_cols[i]]))
    for i in range(1,len(num_cols)):
      loss += tf.keras.losses.MSE(tf.cast(y[num_cols[i]], tf.float32), tf.squeeze(predictions[num_cols[i]]))
    for col in list(cat_cols.keys()):
      loss += tf.keras.losses.SparseCategoricalCrossentropy()(tf.squeeze(y[col]), tf.squeeze(predictions[col]))
  gradients = tape.gradient(loss, model.trainable_variables)
  mean_std = [x.name for x in model.non_trainable_variables if ('batch_norm') in x.name and ('mean' in x.name or 'variance' in x.name)]
  with tf.control_dependencies(mean_std):
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  return(y, predictions, loss)

@tf.function
def test_step(X, y, model):
  predictions = model(X)
  return(y, predictions)

def epoch(data_iter, df, model, optimizer, cat_cols, num_cols, norm_dict):
  out_loss = {}
  out_acc = {}
  n = 0.
  n_batch = 0.
  total_time_dataload = 0.
  total_time_model = 0.
  start_time = time.time()
  for batch in data_iter:
    total_time_dataload += time.time() - start_time
    start_time = time.time()
    t1 = (tf.cast(batch['t1'], tf.float32)-t1_mean)/t1_std
    t2 = (batch['t2']-t2_mean)/t2_std
    ad = batch['ad']
    ad = tf.where(tf.math.is_nan(ad), tf.zeros_like(ad), ad)
    ad = (ad-ad_mean)/ad_std
    fa = batch['fa']
    fa = tf.where(tf.math.is_nan(fa), tf.zeros_like(fa), fa)
    fa = (fa-fa_mean)/fa_std
    md = batch['md']
    md = tf.where(tf.math.is_nan(md), tf.zeros_like(md), md)
    md = (md-md_mean)/md_std
    rd = batch['rd']
    rd = tf.where(tf.math.is_nan(rd), tf.zeros_like(rd), rd)
    rd = (rd-rd_mean)/rd_std
    subjectid = decoder(batch['subjectid'])
    y = get_labels(df, subjectid, list(cat_cols.keys())+num_cols)
    X = tf.concat([t1, t2], axis=4)
    #X = tf.concat([t1, t2], axis=4)
    if optimizer != None:
      y_true, y_pred, loss = train_step(X, y, model, optimizer, cat_cols, num_cols)
    else:
      y_true, y_pred = test_step(X, y, model)
    out_loss, out_acc = calc_loss_acc(out_loss, out_acc, y_true, y_pred, cat_cols, num_cols, norm_dict)
    n += X.shape[0]
    n_batch += 1
    if (n_batch % 10) == 0:
      print(n_batch)
    total_time_model += time.time() - start_time
    start_time = time.time()
  return (out_loss, out_acc, n, total_time_model, total_time_dataload)

def get_labels(df, subjectid, cols = ['nihtbx_fluidcomp_uncorrected_norm']):
  subjects_df = pd.DataFrame(subjectid)
  result_df = pd.merge(subjects_df, df, left_on=0, right_on='subjectkey', how='left')
  output = {}
  for col in cols:
    output[col] = np.asarray(result_df[col].values)
  return output

def best_val(df_best, df_val, df_test):
  df_best = pd.merge(df_best, df_val, how='left', left_on='name', right_on='name')
  df_best = pd.merge(df_best, df_test, how='left', left_on='name', right_on='name')
  df_best.loc[df_best['best_loss_val']>=df_best['cur_loss_val'], 'best_loss_test'] = df_best.loc[df_best['best_loss_val']>=df_best['cur_loss_val'], 'cur_loss_test']
  df_best.loc[df_best['best_loss_val']>=df_best['cur_loss_val'], 'best_loss_val'] = df_best.loc[df_best['best_loss_val']>=df_best['cur_loss_val'], 'cur_loss_val']
  df_best.loc[(df_best['best_acc_val']<=df_best['cur_acc_val'])&(df_best['name'].isin(['female', 'race.ethnicity', 'high.educ_group', 'income_group', 'married'])), 'best_acc_test'] = df_best.loc[(df_best['best_acc_val']<=df_best['cur_acc_val'])&(df_best['name'].isin(['female', 'race.ethnicity', 'high.educ_group', 'income_group', 'married'])), 'cur_acc_test']
  df_best.loc[(df_best['best_acc_val']<=df_best['cur_acc_val'])&(df_best['name'].isin(['female', 'race.ethnicity', 'high.educ_group', 'income_group', 'married'])), 'best_acc_val'] = df_best.loc[(df_best['best_acc_val']<=df_best['cur_acc_val'])&(df_best['name'].isin(['female', 'race.ethnicity', 'high.educ_group', 'income_group', 'married'])), 'cur_acc_val']
  df_best.loc[(df_best['best_acc_val']>=df_best['cur_acc_val'])&(~df_best['name'].isin(['female', 'race.ethnicity', 'high.educ_group', 'income_group', 'married'])), 'best_acc_test'] = df_best.loc[(df_best['best_acc_val']>=df_best['cur_acc_val'])&(~df_best['name'].isin(['female', 'race.ethnicity', 'high.educ_group', 'income_group', 'married'])), 'cur_acc_test']
  df_best.loc[(df_best['best_acc_val']>=df_best['cur_acc_val'])&(~df_best['name'].isin(['female', 'race.ethnicity', 'high.educ_group', 'income_group', 'married'])), 'best_acc_val'] = df_best.loc[(df_best['best_acc_val']>=df_best['cur_acc_val'])&(~df_best['name'].isin(['female', 'race.ethnicity', 'high.educ_group', 'income_group', 'married'])), 'cur_acc_val']
  df_best = df_best.drop(['cur_loss_val', 'cur_acc_val', 'cur_loss_test', 'cur_acc_test'], axis=1)
  return(df_best)

In [12]:
decoder = np.vectorize(lambda x: x.decode('UTF-8'))
template = 'Epoch {0}, Loss: {1:.3f}, Accuracy: {2:.3f}, Val Loss: {3:.3f}, Val Accuracy: {4:.3f}, Time Model: {5:.3f}, Time Data: {6:.3f}'
for col in [0]:
  log_textfile(path_output + modelname + 'multitask_test' + '.log', cat_cols),
  log_textfile(path_output + modelname + 'multitask_test' + '.log', num_cols)
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
  optimizer = tf.keras.optimizers.Adam(lr = 0.001)
  model = Model(cat_cols, num_cols)
  df_best = None
  for e in range(20):
    log_textfile(path_output + modelname + 'multitask_test' + '.log', 'Epochs: ' + str(e))
    loss = tf.Variable(0.)
    acc = tf.Variable(0.) 
    val_loss = tf.Variable(0.)
    val_acc = tf.Variable(0.)
    test_loss = tf.Variable(0.)
    test_acc = tf.Variable(0.)
    tf.keras.backend.set_learning_phase(True)
    train_out_loss, train_out_acc, n, time_model, time_data = epoch(train_iter, train_df, model, optimizer, cat_cols, num_cols, norm_dict)
    tf.keras.backend.set_learning_phase(False)
    val_out_loss, val_out_acc, n, _, _ = epoch(val_iter, val_df, model, None, cat_cols, num_cols, norm_dict)
    test_out_loss, test_out_acc, n, _, _ = epoch(test_iter, test_df, model, None, cat_cols, num_cols, norm_dict)
    loss, acc, _ = format_output(train_out_loss, train_out_acc, n, list(cat_cols.keys())+num_cols)
    val_loss, val_acc, df_val = format_output(val_out_loss, val_out_acc, n, list(cat_cols.keys())+num_cols, print_bl=False)
    test_loss, test_acc, df_test = format_output(test_out_loss, test_out_acc, n, list(cat_cols.keys())+num_cols, print_bl=False)
    df_val.columns = ['name', 'cur_loss_val', 'cur_acc_val']
    df_test.columns = ['name', 'cur_loss_test', 'cur_acc_test']
    if e == 0:
      df_best = pd.merge(df_test, df_val, how='left', left_on='name', right_on='name')
      df_best.columns = ['name', 'best_loss_test', 'best_acc_test', 'best_loss_val', 'best_acc_val']
    df_best = best_val(df_best, df_val, df_test)
    print(df_best[['name', 'best_loss_test', 'best_acc_test']])
    print(df_best[['name', 'best_loss_val', 'best_acc_val']])
    log_textfile(path_output +  modelname + 'multitask_test' + '.log', template.format(e, loss, acc, val_loss, val_acc, time_model, time_data))
    if e in [10, 15]:
      optimizer.lr = optimizer.lr/3
      log_textfile(path_output +  modelname + 'multitask_test' + '.log', 'Learning rate: ' + str(optimizer.lr))
    df_best.to_csv(path_output +  modelname + 'multitask_test' + '.csv')
    #model.save_weights(path_saved + str(e) + "/checkpoint")

{'race.ethnicity': 5, 'married': 6, 'high.educ_group': 4, 'female': 2, 'income_group': 8}
['BMI_norm', 'age_norm', 'vol_norm', 'weight_norm', 'height_norm', 'nihtbx_fluidcomp_uncorrected_norm', 'nihtbx_cryst_uncorrected_norm', 'nihtbx_pattern_uncorrected_norm', 'nihtbx_picture_uncorrected_norm', 'nihtbx_list_uncorrected_norm', 'nihtbx_flanker_uncorrected_norm', 'nihtbx_picvocab_uncorrected_norm', 'nihtbx_cardsort_uncorrected_norm', 'nihtbx_totalcomp_uncorrected_norm', 'nihtbx_reading_uncorrected_norm']
Epochs: 0


10.0


20.0


30.0


40.0


...

60.0


70.0


10.0


10.0


                                 name  best_loss_test  best_acc_test
0                             married        0.116645       0.860656
1                              female        0.054127       0.836066
2                     high.educ_group        0.049467       0.926230
3                        income_group        0.269769       0.237705
4                      race.ethnicity        0.086604       0.778689
5                            BMI_norm        0.284011       4.993375
6                            age_norm        1.394315      77.994365
7                            vol_norm        0.417436       0.418856
8                         weight_norm        0.265595     144.459289
9                         height_norm        1.225225      12.178583
10  nihtbx_fluidcomp_uncorrected_norm        0.868677       0.651561
11      nihtbx_cryst_uncorrected_norm        1.061623       0.653428
12    nihtbx_pattern_uncorrected_norm        0.931524       0.838334
13    nihtbx_picture_uncorrected_n

In [13]:
error

NameError: name 'error' is not defined

In [ ]:
batch = next(iter(train_iter))

In [ ]:
t1 = (tf.cast(batch['t1'], tf.float32)-t1_mean)/t1_std
t2 = (batch['t2']-t2_mean)/t2_std
ad = batch['ad']
ad = tf.where(tf.math.is_nan(ad), tf.zeros_like(ad), ad)
ad = (ad-ad_mean)/ad_std
fa = batch['fa']
fa = tf.where(tf.math.is_nan(fa), tf.zeros_like(fa), fa)
fa = (fa-fa_mean)/fa_std
md = batch['md']
md = tf.where(tf.math.is_nan(md), tf.zeros_like(md), md)
md = (md-md_mean)/md_std
rd = batch['rd']
rd = tf.where(tf.math.is_nan(rd), tf.zeros_like(rd), rd)
rd = (rd-rd_mean)/rd_std
#subjectid = decoder(batch['subjectid'])
#y = get_labels(df, subjectid, list(cat_cols.keys())+num_cols)
#X = tf.concat([t1, t2, ad, fa, md, rd], axis=4)
X = tf.concat([t1, t2], axis=4)    

In [ ]:
tf.keras.backend.set_learning_phase(True)
model(X)['female']

In [ ]:
tf.keras.backend.set_learning_phase(False)
model(X)['female']

In [ ]:
mean_std = [x.name for x in model.non_trainable_variables if ('batch_norm') in x.name and ('mean' in x.name or 'variance' in x.name)]

In [ ]:
model = Model(cat_cols, num_cols)

In [ ]:
model.non_trainable_variables 